In [4]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
%matplotlib inline

In [5]:
data = pd.read_csv("Iris.csv", index_col = False)

In [6]:
c = float(input("Enter penalty for misclassification: "))

Enter penalty for misclassification: 10


In [22]:
data.head()

,A,B,C,D,Label,Label.1
0,5.1,3.5,1.4,0.2,Iris-setosa,1
1,4.9,3.0,1.4,0.2,Iris-setosa,1
2,4.7,3.2,1.3,0.2,Iris-setosa,1
3,4.6,3.1,1.5,0.2,Iris-setosa,1
4,5.0,3.6,1.4,0.2,Iris-setosa,1


In [19]:
v = len(data.columns)-2

In [41]:
o = 2

In [20]:
u = data.columns[-1]
classes = np.unique(data[u])

In [21]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2)
k = len(train) # number of points (also number of constraints)
y = np.array(train[u])
points = []
for i in range(k):
    points.append(np.array(train.iloc[i, :v]))  # Set of points

In [22]:
points[0]

array([6.9, 3.1, 5.4, 2.1], dtype=object)

In [23]:
H = [] # set of hyperplanes
for z in classes:
    s = 'Label w.r.t to class ' + str(z)
    train[s] = np.where(train[u] == z, 1, -1) # ADD distinction column for each class 1 if x in class else -1.

<ipython-input-23-be0e2d4d1650>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[s] = np.where(train[u] == z, 1, -1) # ADD distinction column for each class 1 if x in class else -1.
<ipython-input-23-be0e2d4d1650>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[s] = np.where(train[u] == z, 1, -1) # ADD distinction column for each class 1 if x in class else -1.
<ipython-input-23-be0e2d4d1650>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [24]:
train.head()

,A,B,C,D,Label,Label.1,Label w.r.t to class 1,Label w.r.t to class 2,Label w.r.t to class 3
139,6.9,3.1,5.4,2.1,Iris-virginica,3,-1,-1,1
64,5.6,2.9,3.6,1.3,Iris-versicolor,2,-1,1,-1
105,7.6,3.0,6.6,2.1,Iris-virginica,3,-1,-1,1
77,6.7,3.0,5.0,1.7,Iris-versicolor,2,-1,1,-1
36,5.5,3.5,1.3,0.2,Iris-setosa,1,1,-1,-1


In [25]:
H= []
for z in classes:
    s = 'Label w.r.t to class ' + str(z)
    y_i = np.array(train[s]) 
    # Binary SVM  
    #Dual
    m = gp.Model()
    lambda_i = m.addVars(k, vtype=GRB.CONTINUOUS, name = 'lambda', lb = 0, ub = c)
    m.addConstr(gp.quicksum(lambda_i[i]*y_i[i] for i in range(k)) == 0)
    objective =  gp.quicksum(lambda_i[i] for i in range(k)) - gp.quicksum(gp.quicksum(lambda_i[i]*lambda_i[j]*y_i[i]*y_i[j]*K_linear(points[i], points[j]) for j in range(k)) for i in range(k))/2 

    m.setObjective(objective, GRB.MAXIMIZE)
    m.optimize()
    lambda_final = []
    for i in m.getVars():
        lambda_final.append(i.X)
    lambda_final = np.array(lambda_final)
    a = np.add(y_i[0]*lambda_final[0]*points[0], y_i[1]*lambda_final[1]*points[1])
    for i in range(2, k):
        a = np.add(a, y_i[i]*lambda_final[i]*points[i])
    i = 0
    while lambda_final[i] < 0.0001:
        i = i + 1
    b = y_i[i] - np.sum(points[i]*a)
    H.append([a, b])

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 1 rows, 120 columns and 120 nonzeros
Model fingerprint: 0x2acdc2cc
Model has 7260 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  QObjective range [3e+01, 2e+02]
  Bounds range     [1e+01, 1e+01]
  RHS range        [0e+00, 0e+00]
Presolve time: 0.03s
Presolved: 1 rows, 120 columns, 120 nonzeros
Presolved model has 7260 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 Free vars  : 4
 AA' NZ     : 1.000e+01
 Factor NZ  : 1.500e+01
 Factor Ops : 5.500e+01 (less than 1 second per iteration)
 Threads    : 1

                  Objective                Residual
Iter       Primal          Dual         Primal    Dual     Compl     Time
   0   4.80051532e+05  3.00600000e+05  3.60e+04 1.41e-07  1.00e+06     0s
   1   1.03460896e+03  2.99170743e+05  7.37e+0

In [30]:
H

[[array([-0.04603498908696231, 0.5217227751029782, -1.003164343663556,
         -0.46418022201855336], dtype=object),
  1.4505627817410018],
 [array([-0.04909180148408865, -2.160039273468687, 1.2174766812598463,
         -2.7098674520774324], dtype=object),
  6.678939616535303],
 [array([-1.129438716644949, -1.1294387143048916, 3.4295532602511125,
         4.352806416170655], dtype=object),
  -14.591981664309841]]

In [14]:
def K_linear(x, y):
    return np.sum(x*y)

In [15]:
def K_polynomial(x, y, constant = 1, power = 1):
    return (np.sum(x*y) + constant)**power

In [16]:
def K_RBF(x, y, parameter = 1, constant = 1, order = 2):
    c = np.linalg.norm(x-y, ord = order)
    return np.exp(-parameter*c**2 + constant)

In [42]:
points_test = []
for i in range(len(test)):
    points_test.append(np.array(test.iloc[i, :-o]))

In [50]:
def classifier(H):
    y_predicted = []
    for i in range(len(test)):
        l = []
        for j in range(len(H)):
            l.append(np.sum(H[j][0]*points_test[i]) + H[j][1])
        y_predicted.append(l.index(max(l)) + 1)
    y_predicted = pd.Series(y_predicted, name = 'Predicted')
    return y_predicted

In [44]:
y_actual = pd.Series(test[u], name = 'Actual')

In [51]:
y_predicted = classifier(H)

In [53]:
import sklearn
cf = sklearn.metrics.confusion_matrix(y_actual, y_predicted)

In [54]:
cf

array([[6, 2, 0],
       [0, 9, 0],
       [0, 5, 8]], dtype=int64)

In [67]:
accuracy = 0
for i in range(cf.shape[0]):
    accuracy += cf[i, i]
accuracy = accuracy*100/len(points_test)
print("The accuracy is ",accuracy,"%")

The accuracy is  76.66666666666667 %


In [57]:
len(points_test)

30

In [59]:
(6+9+8)/30

0.7666666666666667